In [37]:
import cv2
import numpy as np
import os
import time

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler

In [2]:
DATA_PATH = 'MP_DATA/'

# Classes/Ações que queremos detectar: (Podemos expandir isso!!)
actions = np.array([chr(i) for i in range(ord('A'), ord('I') + 1)])
 
number_of_videos = 30 # Número de sequências ou vídeos que iremos coletar os frames (Pode ser modificado!)
sequence_length = 180 # Para cada vídeo, iremos coletar 30 frames. (Pode ser modificado!)

### Preprocessamento:

In [3]:
# Dictonary comprehension para relacionar o número ao texto da classe:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(number_of_videos):
        janela = []
        for numero_frame in range(sequence_length):
            res = np.load(DATA_PATH +  action + '/' +str(sequence) + '/' + "{}.npy".format(numero_frame))
            janela.append(res)
            
        # Juntando as sequências:
        sequences.append(janela)

        # Juntando o respectivo label da sequência:
        labels.append(label_map[action])

In [4]:
# Transformando as features e o target em arrays:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [5]:
# Separando em conjunto de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
# Reshaping data:
reshaped_train = np.reshape(X_train, (X_train.shape[0]*X_train.shape[1], X_train.shape[2]))
reshaped_test = np.reshape(X_test, (X_test.shape[0] * X_test.shape[1], X_test.shape[2]))

# Normalization:
scaler = StandardScaler()
normalized_train = scaler.fit_transform(reshaped_train)
normalized_test = scaler.transform(reshaped_test)

X_train_std = np.reshape(normalized_train, X_train.shape)
X_test_std = np.reshape(normalized_test, X_test.shape)

In [7]:
X_train.shape

(216, 180, 1662)

### Treinamento

In [8]:
def get_run_logs(root):
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root, run_id)

In [51]:
# Arquitetura do modelo:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(180, 1662)))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(BatchNormalization())

model.add(LSTM(64, return_sequences=False, activation='relu')) # return_sequences False, pois não iremos retornar as sequencias para a camada Fully connected.

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(9, activation='softmax'))

In [52]:
# Learning schedule:
def exponential_decay_schedule(initial_lr, decay_rate, decay_steps):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_lr,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=False  # Set to True if you want to apply decay at discrete intervals
    )
    return lr_schedule

# Example usage:
initial_lr = 0.01
decay_rate = 0.5
decay_steps = 1000
lr_schedule = exponential_decay_schedule(initial_lr, decay_rate, decay_steps)

In [53]:
# Compilar o modelo
learning_rate = 0.001  # Defina a taxa de aprendizado desejada
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
# Diretório do relatório do treinamento:
root_dir = os.path.join(os.curdir, "my_logs")
run = get_run_logs(root_dir)

# Callback do tensorboard:
tb_callback = TensorBoard(log_dir=run)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Treinar o modelo
batch = 32
model.fit(X_train, y_train, epochs=1000, validation_split=0.2, callbacks=[tb_callback, early_stopping], batch_size=batch)

Epoch 1/1000
6/6 [==============================] - 12s 653ms/step - loss: 2.7138 - accuracy: 0.0872 - val_loss: 7.2691 - val_accuracy: 0.1136
Epoch 2/1000
6/6 [==============================] - 3s 488ms/step - loss: 2.7093 - accuracy: 0.1453 - val_loss: 45.4354 - val_accuracy: 0.0909
Epoch 3/1000
6/6 [==============================] - 3s 515ms/step - loss: 2.5596 - accuracy: 0.1512 - val_loss: 54.0150 - val_accuracy: 0.1591
Epoch 4/1000
6/6 [==============================] - 3s 600ms/step - loss: 2.6788 - accuracy: 0.0814 - val_loss: 2.1454 - val_accuracy: 0.1818
Epoch 5/1000
6/6 [==============================] - 3s 513ms/step - loss: 2.5472 - accuracy: 0.1337 - val_loss: 2.1935 - val_accuracy: 0.1591
Epoch 6/1000
6/6 [==============================] - 3s 500ms/step - loss: 2.6301 - accuracy: 0.1105 - val_loss: 2.1929 - val_accuracy: 0.1591
Epoch 7/1000
6/6 [==============================] - 3s 595ms/step - loss: 2.6062 - accuracy: 0.0988 - val_loss: 2.1989 - val_accuracy: 0.1136
Epo

In [ ]:
# Resumno sobre os paámetros treináveis do modelo:
model.summary()

In [ ]:
# Predições do modelo no conjunto de teste:
resultado = model.predict(X_test)

# Criando uma lista com o index da classe predita e da classe real:
y_pred = [np.argmax(line) for line in resultado]
y_test_pos = [np.argmax(line) for line in y_test]

In [ ]:
# Calculando acurácia no conjunto de teste:
acuracia = accuracy_score(y_test_pos, y_pred)
print(f"Acurácia - Test: {acuracia}")

In [ ]:
# Salvando o modelo
model.save(r'modelos\modelo6.keras')